In [9]:
import numpy
import sys
sys.path.append('../../main/regression/')
import scipy.io

#### To Be Specified
filename = 'UB_n=100000_d=500.mat' 

# load data
dataname = filename[0:2]
dataDict = scipy.io.loadmat(filename)
matX = dataDict['matX']
vecW = dataDict['vecW']
print(matX.shape)


(100000, 500)


In [10]:
import numpy
import sys
sys.path.append('../../main/sketch/')
import countsketch as csketch
import srft


def solveW(vecS, matV, matUY, ngamma):
    '''
    Input
        X = U * diag(S) * V
        UY = U^T * Y
        ngamma = n * gamma
    '''
    vec1 = ngamma / vecS
    vec1 = vec1 + vecS
    mat1 = matUY / vec1.reshape(len(vec1), 1)
    matW = numpy.dot(matV.T, mat1)
    return matW

def solveWhat(vecS, matV, matXY, ngamma):
    vec1 = vecS * vecS + ngamma
    mat1 = numpy.dot(matV, matXY)
    mat1 = mat1 / vec1.reshape(len(vec1), 1)
    matW = numpy.dot(matV.T, mat1)
    return matW

def objFunValOpt(matX, matY, vecS, matV, matUY, vecGamma):
    n = matX.shape[0]
    lenGamma = len(vecGamma)
    vecObjOpt = numpy.zeros(lenGamma)
    for i in range(lenGamma):
        gamma = vecGamma[i]
        matW = solveW(vecS, matV, matUY, n * gamma)
        matRes = numpy.dot(matX, matW) - matY
        err = numpy.linalg.norm(matRes)
        del matRes
        reg = numpy.linalg.norm(matW)
        del matW
        vecObjOpt[i] = err * err / n + gamma * reg * reg
    return vecObjOpt
    


def objFunValSketch(matX, matY, matXY, vecGamma, sketch, s, vecLev):
    repeat = 10
    n, d = matX.shape
    lenGamma = len(vecGamma)
    matObjTilde = numpy.zeros((lenGamma, repeat))
    matObjHat = numpy.zeros((lenGamma, repeat))
    
    
    for j in range(repeat):
        vecObjTildeTmp = numpy.zeros(lenGamma)
        vecObjHatTmp = numpy.zeros(lenGamma)

        # ================== sketching ================== #
        if sketch == 'uni':
            idx = numpy.random.choice(n, s, replace=False)
            matXsketch = matX[idx, :] * numpy.sqrt(n/s)
            matYsketch = matY[idx, :] * numpy.sqrt(n/s)
        elif sketch == 'lev':
            prob = vecLev / numpy.sum(vecLev)
            scaling = 1 / numpy.sqrt(s * prob)
            scaling = scaling.reshape(n, 1)
            idx = numpy.random.choice(n, s, replace=False, p=prob)
            matXsketch = matX[idx, :] * scaling[idx]
            matYsketch = matY[idx, :] * scaling[idx]
        elif sketch == 'shrink':
            prob = vecLev / numpy.sum(vecLev)
            prob = (prob + 1/n) / 2
            scaling = 1 / numpy.sqrt(s * prob)
            scaling = scaling.reshape(n, 1)
            idx = numpy.random.choice(n, s, replace=False, p=prob)
            matXsketch = matX[idx, :] * scaling[idx]
            matYsketch = matY[idx, :] * scaling[idx]
        elif sketch == 'gauss':
            matSketch = numpy.random.randn(s, n) / numpy.sqrt(s) 
            matXsketch = numpy.dot(matSketch, matX)
            matYsketch = numpy.dot(matSketch, matY)
        elif sketch == 'srft':
            matXsketch, matYsketch = srft.srft2(matX, matY, s)
        elif sketch == 'count':
            matXsketch, matYsketch = csketch.countsketch(matX, matY, s)


        matU, vecS, matV = numpy.linalg.svd(matXsketch, full_matrices=False)
        matUY = numpy.dot(matU.T, matYsketch)

        # ================ compute Wtilde ================ #
        for i in range(lenGamma):
            gamma = vecGamma[i]
            matWtilde = solveW(vecS, matV, matUY, n * gamma)
            matRes = numpy.dot(matX, matWtilde) - matY
            err = numpy.linalg.norm(matRes)
            del matRes
            reg = numpy.linalg.norm(matWtilde)
            del matWtilde
            vecObjTildeTmp[i] = err * err / n + gamma * reg * reg

        # ================ compute What ================ #
        for i in range(lenGamma):
            gamma = vecGamma[i]
            matWhat = solveWhat(vecS, matV, matXY, n * gamma)
            #matWhat = numpy.zeros((d, 1))
            matRes = numpy.dot(matX, matWhat) - matY
            err = numpy.linalg.norm(matRes)
            del matRes
            reg = numpy.linalg.norm(matWhat)
            del matWhat
            vecObjHatTmp[i] = err * err / n + gamma * reg * reg
        matObjTilde[:, j] = vecObjTildeTmp
        matObjHat[:, j] = vecObjHatTmp
    vecObjTilde = numpy.mean(matObjTilde, axis=1)
    vecObjHat = numpy.mean(matObjHat, axis=1)
    return vecObjTilde, vecObjHat


def objExperiment(matX, matW, s, vecGamma, vecXi):
    n, d = matX.shape
    lenGamma = len(vecGamma)
    lenXi = len(vecXi)
    objOpt = numpy.zeros((lenGamma, lenXi))
    objTildeUni = numpy.zeros((lenGamma, lenXi))
    objHatUni = numpy.zeros((lenGamma, lenXi))
    objTildeLev = numpy.zeros((lenGamma, lenXi))
    objHatLev = numpy.zeros((lenGamma, lenXi))
    objTildeShrink = numpy.zeros((lenGamma, lenXi))
    objHatShrink = numpy.zeros((lenGamma, lenXi))
    objTildeGauss = numpy.zeros((lenGamma, lenXi))
    objHatGauss = numpy.zeros((lenGamma, lenXi))
    objTildeSrft = numpy.zeros((lenGamma, lenXi))
    objHatSrft = numpy.zeros((lenGamma, lenXi))
    objTildeCount = numpy.zeros((lenGamma, lenXi))
    objHatCount = numpy.zeros((lenGamma, lenXi))
    
    matU, vecS, matV = numpy.linalg.svd(matX, full_matrices=False)
    vecLev = numpy.sum(matU ** 2, axis=1)
    
    matNoise = numpy.random.randn(n, 1)
    for i in range(lenXi):
        xi = vecXi[i]
        matY = numpy.dot(matX, matW) + matNoise * xi
        matXY = numpy.dot(matX.T, matY)
        matUY = numpy.dot(matU.T, matY)
        # optimal
        print('Doing optimal ridge regression...')
        vecObjOpt = objFunValOpt(matX, matY, vecS, matV, matUY, vecGamma)
        objOpt[:, i] = vecObjOpt
        # uniform
        print('Doing uniform sampling...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'uni', s, vecLev)
        objTildeUni[:, i] = vecObjTilde
        objHatUni[:, i] = vecObjHat
        # leverage
        print('Doing leverage score sampling...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'lev', s, vecLev)
        objTildeLev[:, i] = vecObjTilde
        objHatLev[:, i] = vecObjHat
        # shrinkage leverage
        print('Doing shrinkage leverage score sampling...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'shrink', s, vecLev)
        objTildeShrink[:, i] = vecObjTilde
        objHatShrink[:, i] = vecObjHat
        # Gaussian projection
        print('Doing Gaussian projection...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'gauss', s, vecLev)
        objTildeGauss[:, i] = vecObjTilde
        objHatGauss[:, i] = vecObjHat
        # SRFT
        print('Doing SRFT...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'srft', s, vecLev)
        objTildeSrft[:, i] = vecObjTilde
        objHatSrft[:, i] = vecObjHat
        # Count Sketch
        print('Doing count sketch...')
        vecObjTilde, vecObjHat = objFunValSketch(matX, matY, matXY, vecGamma, 'count', s, vecLev)
        objTildeCount[:, i] = vecObjTilde
        objHatCount[:, i] = vecObjHat
    
    resultDict = {'gamma': vecGamma,
                  'xi': vecXi,
                  'Opt': objOpt,
                 'TildeUni': objTildeUni,
                 'HatUni': objHatUni,
                 'TildeLev': objTildeLev,
                 'HatLev': objHatLev,
                 'TildeShrink': objTildeShrink,
                 'HatShrink': objHatShrink,
                 'TildeGauss': objTildeGauss,
                 'HatGauss': objHatGauss,
                 'TildeSrft': objTildeSrft,
                 'HatSrft': objHatSrft,
                 'TildeCount': objTildeCount,
                 'HatCount': objHatCount }
    return resultDict

        
    

In [11]:

# Parameters Fixed for All the Experiments
vecGamma = [0, 1e-12, 1e-11, 1e-10, 3e-10, 1e-9, 3e-9, 1e-8, 2e-8, 5e-8, 1e-7, 2e-7, 5e-7, 1e-6, 2e-6, 5e-6, 1e-5, 3e-5, 1e-4, 1e-3, 1e-2]
vecXi = [0.001, 0.01, 0.1]
s = 5000

outputFileName = 'objFunVal_' + dataname + '.mat'
resultDict = objExperiment(matX, vecW, s, vecGamma, vecXi)
scipy.io.savemat(outputFileName, resultDict)

Doing optimal ridge regression...
Doing uniform sampling...
Doing leverage score sampling...
Doing shrinkage leverage score sampling...
Doing Gaussian projection...
Doing SRFT...
Doing count sketch...
Doing optimal ridge regression...
Doing uniform sampling...
Doing leverage score sampling...
Doing shrinkage leverage score sampling...
Doing Gaussian projection...
Doing SRFT...
Doing count sketch...
Doing optimal ridge regression...
Doing uniform sampling...
Doing leverage score sampling...
Doing shrinkage leverage score sampling...
Doing Gaussian projection...
Doing SRFT...
Doing count sketch...


In [12]:
print(resultDict)

{'Opt': array([[  9.92126750e-07,   9.92126750e-05,   9.92126750e-03],
       [  9.94729389e-07,   9.94600023e-05,   9.94597377e-03],
       [  9.96070180e-07,   9.95033306e-05,   9.95018075e-03],
       [  1.00363651e-06,   9.95533247e-05,   9.95444816e-03],
       [  1.01738580e-06,   9.95857226e-05,   9.95639123e-03],
       [  1.05912078e-06,   9.96454569e-05,   9.95843207e-03],
       [  1.16241073e-06,   9.97618106e-05,   9.96022749e-03],
       [  1.46599297e-06,   1.00076850e-04,   9.96239202e-03],
       [  1.83770650e-06,   1.00454411e-04,   9.96391470e-03],
       [  2.77939312e-06,   1.01403796e-04,   9.96647572e-03],
       [  4.09621047e-06,   1.02729112e-04,   9.96911278e-03],
       [  6.31121897e-06,   1.04958682e-04,   9.97277135e-03],
       [  1.14922401e-05,   1.10170133e-04,   9.98003767e-03],
       [  1.78186209e-05,   1.16525230e-04,   9.98799605e-03],
       [  2.64308758e-05,   1.25170699e-04,   9.99823054e-03],
       [  4.00815159e-05,   1.38880239e-04,   1